In [3]:
import numpy as np
import opt_einsum as oe
from copy import deepcopy
import itertools
from scipy import integrate
import pickle as pk
import copy
import math
import scipy
import pandas as pk

import torch
torch.set_default_dtype(torch.float64)

import os
import sys
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy'))
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy/tnpy'))

import tnpy as tp

import sympy as sp
from sympy import pprint, simplify, expand, latex
from IPython.display import display
from fractions import Fraction
# sp.init_printing (use_latex="mathjax", latex_mode="equation")
sp.init_printing(use_latex='mathjax', latex_mode='equation', fontsize='20pt')


import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits
# from mpl_toolkits import mplot3d
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import numpy as np
import pandas as pd
import os
import itertools
import glob

%matplotlib inline

class ScalarFormatterForceFormatZero(ticker.ScalarFormatter):
    def _set_format(self):  # Override function that finds format to use
        self.format = '%1.0f'  # Give format here

class ScalarFormatterForceFormatOne(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.1f'

class ScalarFormatterForceFormat(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.2f'
        
class ScalarFormatterForceFormatThree(ticker.ScalarFormatter):
    def _set_format(self):
        self.format = '%1.3f'

# Statistical physics primer

# 2D classical Ising model

## TN-rep

### On the link

### On the original lattice

## Exact result for the 2D square Ising model

For a 2D classical Ising model with horizontal coupling $J$ and vertical coupling $J^{\prime}$,
$$
\begin{equation}
    \ln\left(Z\right)=-\beta{F}
    =-\frac{\ln(2)}{2}-\frac{1}{2\pi}\int_{0}^{\pi}\ln\left[\cosh(2K)\cosh(2L)+\frac{1}{k}\sqrt{1+k^{2}-2k\cos(2\theta)}\right]d\theta
\end{equation}
$$
with $k=1/\left[\sinh(2K)\sinh(2K)\right]$.

In [41]:
# analytical results for Ising model
# https://en.wikipedia.org/wiki/Square_lattice_Ising_model

b = sp.Symbol(r'\beta', real=True)
t = sp.Symbol(r'\theta', real=True)
J = sp.Symbol('J', real=True)

# sp.integrate: to compute the integral
# sp.Integral: just create a symbolic integral expression

k = 1/(sp.sinh(2*b*J))**2
e = -sp.log(2)/2-(1/(2*sp.pi))*sp.Integral(sp.log(sp.cosh(2*b*J)**2+(1/k)*sp.sqrt(1+k**2-2*k*sp.cos(2*t))), (t, 0, sp.pi))
# display(e)
# display(e.evalf(subs={b: 0.1, J:1.0}))

def logZ(b):
    k = 1/(sp.sinh(2*b*J))**2
    return -sp.log(2)/2-(1/(2*sp.pi))*sp.Integral(sp.log(sp.cosh(2*b*J)**2+(1/k)*sp.sqrt(1+k**2-2*k*sp.cos(2*t))), (t, 0, sp.pi))

def U(b):
    k = 1/(sp.sinh(2*b*J))**2
    return -J*sp.coth(2*b*J)*(1+(2/sp.pi)*(2*(sp.tanh(2*b*J))**2-1)*sp.Integral(1.0/sp.sqrt(1-4*k/((1+k)**2)*(sp.sin(t))**2), (t, 0, sp.pi/2)))

beta = 0.3
display('Free energy F:', logZ(b))
display(logZ(b).evalf(subs={b:beta, J:1.0}))
display('Internal energy E:', sp.diff(logZ(b), b))
display(sp.diff(logZ(b), b).evalf(subs={b:beta, J:1.0}))

'Free energy F:'

  π                                                                           
  ⌠                                                                           
  ⎮    ⎛     ___________________________________________                      
  ⎮    ⎜    ╱   2⋅cos(2⋅\theta)               1              2                
  ⎮ log⎜   ╱  - ──────────────── + 1 + ──────────────── ⋅sinh (2⋅J⋅\beta) + co
  ⎮    ⎜  ╱         2                      4                                  
  ⎮    ⎝╲╱      sinh (2⋅J⋅\beta)       sinh (2⋅J⋅\beta)                       
  ⌡                                                                           
  0                                                                           
- ────────────────────────────────────────────────────────────────────────────
                                                   2⋅π                        

                                  
                                  
              ⎞                   
  2           ⎟          

-0.790559070951263

'Internal energy E:'

 π                                                                            
 ⌠                                                                            
 ⎮                                                                            
 ⎮                                                                            
 ⎮          ___________________________________________                       
 ⎮         ╱   2⋅cos(2⋅\theta)               1                                
 ⎮ 4⋅J⋅   ╱  - ──────────────── + 1 + ──────────────── ⋅sinh(2⋅J⋅\beta)⋅cosh(2
 ⎮       ╱         2                      4                                   
 ⎮     ╲╱      sinh (2⋅J⋅\beta)       sinh (2⋅J⋅\beta)                        
 ⎮                                                                            
 ⎮                                                                            
 ⎮                                                                            
-⎮ ─────────────────────────────────────────────────

-0.704499070832445

In [3]:
beta, step = 0.1, 0.02
num_sample = 50
betas = np.linspace(beta, beta+step*num_sample, num_sample, endpoint=False)

df = pd.DataFrame(columns=['beta', 'free_ene', 'internal_ene'])
print('beta', 'free energy', 'internal energy')
for beta in betas:
    data = ['{:.2f}'.format(beta), logZ(b).evalf(subs={b:beta, J:1.0}), sp.diff(logZ(b), b).evalf(subs={b:beta, J:1.0})]
    print(data)
    df.loc[len(df)] = data

# df.to_csv('square_classical_ising_exact.csv', index=False)

beta free energy internal energy
['0.10', -0.703231242285832, -0.203377391097356]
['0.12', -0.707722179056315, -0.245871222486703]
['0.14', -0.713072929531680, -0.289391258033900]
['0.16', -0.719306016790442, -0.334139998248643]
['0.18', -0.726448185218975, -0.380337878113544]
['0.20', -0.734530812276326, -0.428228833240348]
['0.22', -0.743590449273009, -0.478087754153874]
['0.24', -0.753669537298383, -0.530230734794032]
['0.26', -0.764817367595539, -0.585029577679236]
['0.28', -0.777091393948020, -0.642933021814817]
['0.30', -0.790559070951263, -0.704499070832446]
['0.32', -0.805300513707733, -0.770446689818235]
['0.34', -0.821412514067894, -0.841743750716971]
['0.36', -0.839014965034055, -0.919769346517525]
['0.38', -0.858262002413707, -1.00664930479848]
['0.40', -0.879363820774948, -1.10607920374579]
['0.42', -0.902639219096205, -1.22605484028068]
['0.44', -0.928728540456889, -1.40222696003143]
['0.46', -0.958871412868292, -1.58117731752430]
['0.48', -0.991524464466794, -1.677722451

In [4]:
class SquareClassicalIsing(object):
    r'''
    square lattice of 2D Ising model
    '''

    def __init__(self, beta: float, J: float, dtype=torch.float64, ctms=None):
        r'''

        Parameters
        ----------
        '''

        self._phys_dim = 2
        self._beta, self._J = beta, J
        
        # bond weight matrix
        w = torch.tensor([[torch.exp(-beta*J), torch.exp(beta*J)], [torch.exp(beta*J), torch.exp(-beta*J)]])
        u, s, v = tp.linalg.svd(w)
        m, mp = u @ torch.sqrt(s).diag(), torch.sqrt(s).diag() @ v

        # build the site tensor
        self._site_tensor = torch.einsum('as,sb,sc,ds->abcd', mp, m, m, mp).to(dtype)

        self._dtype = dtype

        self._ctms = ctms

        if self._ctms is not None:
            assert ctms[0].dtype == dtype, 'CTM tensor dtype is not matched'
            self._rho = self._ctms[0].shape[0]
        else:
            self._rho = 0

    def update_ctms(self, ctms):

        self._ctms = ctms
        self._rho = self._rho = self._ctms[0].shape[0]

        return 1

    def ctmrg_mu(self):

        mps = [self._ctms[2], self._ctms[5], self._ctms[3]]
        mpo = [self._ctms[6], self._site_tensor, self._ctms[7]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ABC,aB->aCA', mpo[0], mps[0])
        mpo_mps[1] = torch.einsum('ABCD,efB->eAfCD', mpo[1], mps[1])
        mpo_mps[2] = torch.einsum('ABC,aB->aCA', mpo[2], mps[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr_0)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl_0, mpo_mps[1], pr_1)
        mps[2] = torch.einsum('abc,bcd->ad', pl_1, mpo_mps[2])

        # update CTM tensors
        self._ctms[2] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[5] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[3] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_mu_sym(self):
        r'''
        CTMRG one step move
        symmetric projectors according to: https://arxiv.org/abs/1402.2859
        '''

        mps = [self._ctms[2], self._ctms[5], self._ctms[3]]
        mpo = [self._ctms[6], self._site_tensor, self._ctms[7]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ABC,aB->aCA', mpo[0], mps[0])
        mpo_mps[1] = torch.einsum('ABCD,efB->eAfCD', mpo[1], mps[1])
        mpo_mps[2] = torch.einsum('ABC,aB->aCA', mpo[2], mps[2])

        # find symmetric projectors
        sym_mpo_mps = [t.clone() for t in mpo_mps]
        sym_mpo_mps.insert(1, mpo_mps[1])

        rs, ls = [], []
        temp = sym_mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[2])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = sym_mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-3], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,bcd->ad', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[2] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[5] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[3] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_mu_test(self):

        mps = [self._ctms[2], self._ctms[5], self._ctms[3]]
        mpo = [self._ctms[6], self._site_tensor, self._ctms[7]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ABC,aB->aCA', mpo[0], mps[0])
        mpo_mps[1] = torch.einsum('ABCD,efB->eAfCD', mpo[1], mps[1])
        mpo_mps[2] = torch.einsum('ABC,aB->aCA', mpo[2], mps[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        pr, pl = pr_1, pl_0

        mps = [None]*3
        mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,bcd->ad', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[2] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[5] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[3] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_md(self):

        mps = [self._ctms[0], self._ctms[4], self._ctms[1]]
        mpo = [self._ctms[6], self._site_tensor, self._ctms[7]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ab,bcd->adc', mps[0], mpo[0])
        mpo_mps[1] = torch.einsum('abc,defc->adbfe', mps[1], mpo[1])
        mpo_mps[2] = torch.einsum('ab,bcd->adc', mps[2], mpo[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr_0)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl_0, mpo_mps[1], pr_1)
        mps[2] = torch.einsum('abc,bcd->ad', pl_1, mpo_mps[2])

        # update CTM tensors
        self._ctms[0] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[4] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[1] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_md_sym(self):

        mps = [self._ctms[0], self._ctms[4], self._ctms[1]]
        mpo = [self._ctms[6], self._site_tensor, self._ctms[7]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ab,bcd->adc', mps[0], mpo[0])
        mpo_mps[1] = torch.einsum('abc,defc->adbfe', mps[1], mpo[1])
        mpo_mps[2] = torch.einsum('ab,bcd->adc', mps[2], mpo[2])

        # find the symmetric projectors
        sym_mpo_mps = [t.clone() for t in mpo_mps]
        sym_mpo_mps.insert(1, mpo_mps[1])

        rs, ls = [], []
        temp = sym_mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[2])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = sym_mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-3], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,bcd->ad', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[0] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[4] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[1] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_md_test(self):

        mps = [self._ctms[0], self._ctms[4], self._ctms[1]]
        mpo = [self._ctms[6], self._site_tensor, self._ctms[7]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ab,bcd->adc', mps[0], mpo[0])
        mpo_mps[1] = torch.einsum('abc,defc->adbfe', mps[1], mpo[1])
        mpo_mps[2] = torch.einsum('ab,bcd->adc', mps[2], mpo[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        pr, pl = pr_1, pl_0

        mps = [None]*3
        mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,bcd->ad', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[0] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[4] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[1] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_ml(self):

        mps = [self._ctms[0], self._ctms[6], self._ctms[2]]
        mpo = [self._ctms[4], self._site_tensor, self._ctms[5]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ab,acd->cbd', mps[0], mpo[0])
        mpo_mps[1] = torch.einsum('abc,cdef->afbde', mps[1], mpo[1])
        mpo_mps[2] = torch.einsum('ab,acd->cbd', mps[2], mpo[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,bcd->ad', mpo_mps[0], pr_0)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl_0, mpo_mps[1], pr_1)
        mps[2] = torch.einsum('abc,dbc->da', pl_1, mpo_mps[2])

        # update CTM tensors
        self._ctms[0] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[6] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[2] = mps[2] / torch.linalg.norm(mps[2])
        
        return 1


    def ctmrg_ml_sym(self):

        mps = [self._ctms[0], self._ctms[6], self._ctms[2]]
        mpo = [self._ctms[4], self._site_tensor, self._ctms[5]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ab,acd->cbd', mps[0], mpo[0])
        mpo_mps[1] = torch.einsum('abc,cdef->afbde', mps[1], mpo[1])
        mpo_mps[2] = torch.einsum('ab,acd->cbd', mps[2], mpo[2])

        # find the symmetric projectors
        sym_mpo_mps = [t.clone() for t in mpo_mps]
        sym_mpo_mps.insert(1, mpo_mps[1])

        rs, ls = [], []
        temp = sym_mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[2])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = sym_mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-3], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,bcd->ad', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,dbc->da', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[0] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[6] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[2] = mps[2] / torch.linalg.norm(mps[2])
        
        return 1


    def ctmrg_ml_test(self):

        mps = [self._ctms[0], self._ctms[6], self._ctms[2]]
        mpo = [self._ctms[4], self._site_tensor, self._ctms[5]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('ab,acd->cbd', mps[0], mpo[0])
        mpo_mps[1] = torch.einsum('abc,cdef->afbde', mps[1], mpo[1])
        mpo_mps[2] = torch.einsum('ab,acd->cbd', mps[2], mpo[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        pr, pl = pr_1, pl_0

        mps = [None]*3
        mps[0] = torch.einsum('abc,bcd->ad', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,dbc->da', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[0] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[6] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[2] = mps[2] / torch.linalg.norm(mps[2])
        
        return 1


    def ctmrg_mr(self):

        mps = [self._ctms[1], self._ctms[7], self._ctms[3]]
        mpo = [self._ctms[4], self._site_tensor, self._ctms[5]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('abc,bd->adc', mpo[0], mps[0])
        mpo_mps[1] = torch.einsum('abcd,efc->edfba', mpo[1], mps[1])
        mpo_mps[2] = torch.einsum('abc,bd->adc', mpo[2], mps[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,bcd->ad', mpo_mps[0], pr_0)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl_0, mpo_mps[1], pr_1)
        mps[2] = torch.einsum('abc,dbc->da', pl_1, mpo_mps[2])

        # update CTM tensors
        self._ctms[1] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[7] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[3] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_mr_sym(self):

        mps = [self._ctms[1], self._ctms[7], self._ctms[3]]
        mpo = [self._ctms[4], self._site_tensor, self._ctms[5]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('abc,bd->adc', mpo[0], mps[0])
        mpo_mps[1] = torch.einsum('abcd,efc->edfba', mpo[1], mps[1])
        mpo_mps[2] = torch.einsum('abc,bd->adc', mpo[2], mps[2])

        # find the symmetric projectors
        sym_mpo_mps = deepcopy(mpo_mps)
        sym_mpo_mps.insert(1, mpo_mps[1])

        rs, ls = [], []
        temp = sym_mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[2])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = sym_mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-3], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag().to(self._dtype)
        # projectors
        pr = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        mps = [None]*3
        mps[0] = torch.einsum('abc,bcd->ad', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,dbc->da', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[1] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[7] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[3] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctmrg_mr_test(self):

        mps = [self._ctms[1], self._ctms[7], self._ctms[3]]
        mpo = [self._ctms[4], self._site_tensor, self._ctms[5]]

        mpo_mps = [None]*3
        mpo_mps[0] = torch.einsum('abc,bd->adc', mpo[0], mps[0])
        mpo_mps[1] = torch.einsum('abcd,efc->edfba', mpo[1], mps[1])
        mpo_mps[2] = torch.einsum('abc,bd->adc', mpo[2], mps[2])

        rs, ls = [], []
        temp = mpo_mps[0]
        q, r = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 0))
        rs.append(r)
        temp = torch.einsum('abc,bcdef->adef', r, mpo_mps[1])
        q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
        rs.append(r)

        temp = mpo_mps[-1]
        q, l = tp.linalg.tqr(temp, group_dims=((0,), (1, 2)), qr_dims=(1, 2))
        ls.append(l)
        temp = torch.einsum('abcde,cdf->abfe', mpo_mps[-2], l)
        q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
        ls.append(l)

        ls.reverse()

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[0], ls[0]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_0 = torch.einsum('abc,cd->abd', ls[0], vt_dagger @ sst_inv)
        pl_0 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[0])

        u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
        ut, st, vt = u[:, :self._rho], s[:self._rho], v[:self._rho, :]
        ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
        # inverse of square root
        sst_inv = (1.0 / torch.sqrt(st)).diag()
        # projectors
        pr_1 = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
        pl_1 = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

        pr, pl = pr_1, pl_0

        mps = [None]*3
        mps[0] = torch.einsum('abc,bcd->ad', mpo_mps[0], pr)
        mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
        mps[2] = torch.einsum('abc,dbc->da', pl, mpo_mps[2])

        # update CTM tensors
        self._ctms[1] = mps[0] / torch.linalg.norm(mps[0])
        self._ctms[7] = mps[1] / torch.linalg.norm(mps[1])
        self._ctms[3] = mps[2] / torch.linalg.norm(mps[2])

        return 1


    def ctm_mag(self):

        pass


    def parf(self):

        env_l = torch.einsum('ab,bcd,ec->ade', self._ctms[0], self._ctms[6], self._ctms[2])
        env_r = torch.einsum('ab,bcd,ec->ade', self._ctms[1], self._ctms[7], self._ctms[3])

        temp = env_l.clone()
        temp = torch.einsum('abc,ade,bfge,chf->dgh', temp, self._ctms[4], self._site_tensor, self._ctms[5])

        return torch.einsum('abc,abc', temp, env_r)

    def ctm_bond_energy(self):

        # pure bond weight matrix
        w = [
            [torch.exp(-self._beta*self._J), torch.exp(self._beta*self._J)],
            [torch.exp(self._beta*self._J), torch.exp(-self._beta*self._J)],
            ]
        u, s, v = tp.linalg.svd(torch.tensor(w))
        pure_m, pure_mp = u @ torch.sqrt(s).diag(), torch.sqrt(s).diag() @v

        # impure bond weight matrix
        w = [
            [self._J*torch.exp(-self._beta*self._J), -self._J*torch.exp(self._beta*self._J)],
            [-self._J*torch.exp(self._beta*self._J), self._J*torch.exp(-self._beta*self._J)],
            ]
        u, s, v = tp.linalg.svd(torch.tensor(w))
        impure_m, impure_mp = u @ torch.sqrt(s).diag(), torch.sqrt(s).diag() @ v

        pure_tens = [self._site_tensor]*2

        impure_tens = [None]*2
        impure_tens[0] = torch.einsum('as,sb,sc,ds->abcd', pure_mp, pure_m, impure_m, pure_mp).to(self._dtype)
        impure_tens[1] = torch.einsum('as,sb,sc,ds->abcd', impure_mp, pure_m, pure_m, pure_mp).to(self._dtype)

        env_l = torch.einsum('ab,bcd,ec->ade', self._ctms[0], self._ctms[6], self._ctms[2])
        env_r = torch.einsum('ab,bcd,ec->ade', self._ctms[1], self._ctms[7], self._ctms[3])

        # den
        temp = env_l.clone()
        temp = torch.einsum('abc,ade,bfge,chf->dgh', temp, self._ctms[4], pure_tens[0], self._ctms[5])
        temp = torch.einsum('abc,ade,bfge,chf->dgh', temp, self._ctms[4], pure_tens[1], self._ctms[5])
        den = torch.einsum('abc,abc', temp, env_r)

        # num
        temp = env_l.clone()
        temp = torch.einsum('abc,ade,bfge,chf->dgh', temp, self._ctms[4], impure_tens[0], self._ctms[5])
        temp = torch.einsum('abc,ade,bfge,chf->dgh', temp, self._ctms[4], impure_tens[1], self._ctms[5])
        num = torch.einsum('abc,abc', temp, env_r)

        return num / den

In [5]:
rho = 4

# cs = [torch.rand(rho, rho)+1.j*torch.rand(rho, rho) for i in range(4)]
# es = [torch.rand(rho, rho, 2)+1.j*torch.rand(rho, rho, 2) for i in range(4)]
cs = [torch.rand(rho, rho) for i in range(4)]
es = [torch.rand(rho, rho, 2) for i in range(4)]
ctms = cs+es

beta, step = 0.2, 0.1
num_sample = 1
betas = np.linspace(beta, beta+step*num_sample, num_sample, endpoint=False)

ising = SquareClassicalIsing(beta=torch.tensor(beta), J=1.0, dtype=torch.float64)
ising.update_ctms(ctms)

print(ising._rho)
print(ising.ctm_bond_energy())

4
tensor(-0.30969)


In [6]:
num = 32

for l in range(num):

    ising.ctmrg_mu_sym()
    ising.ctmrg_md_sym()
    ising.ctmrg_ml_sym()
    ising.ctmrg_mr_sym()

    bond_ene = ising.ctm_bond_energy()
    print(l, bond_ene.item()*2)

0 -0.41824220657874883
1 -0.4274622504598082
2 -0.4281839447270691
3 -0.4282261288374566
4 -0.42822866339756954
5 -0.4282288220041636
6 -0.428228832465865
7 -0.42822883318369204
8 -0.42822883323453087
9 -0.4282288332382192
10 -0.4282288332384912
11 -0.4282288332385123
12 -0.4282288332385143
13 -0.42822883323851446
14 -0.42822883323851446
15 -0.42822883323851446
16 -0.42822883323851446
17 -0.42822883323851446
18 -0.4282288332385146
19 -0.42822883323851446
20 -0.42822883323851463
21 -0.4282288332385147
22 -0.4282288332385146
23 -0.4282288332385146
24 -0.42822883323851446
25 -0.42822883323851463
26 -0.4282288332385147
27 -0.42822883323851446
28 -0.42822883323851463
29 -0.4282288332385145
30 -0.42822883323851463
31 -0.42822883323851463


In [97]:
beta = 0.5
beta = torch.tensor([beta]).requires_grad_(True)
print(beta.shape, beta)

ising = SquareClassicalIsing(beta=beta, J=1.0)

rho = 8

cs = [torch.rand(rho, rho).requires_grad_(True) for i in range(4)]
es = [torch.rand(rho, rho, 2).requires_grad_(True) for i in range(4)]
ctms = cs+es

ising.update_ctms(ctms)

torch.Size([1]) tensor([0.50000], requires_grad=True)


1

In [98]:
def ctmrg_mu_sym(site_tensor, ctms):
    r'''
    CTMRG one step move
    symmetric projectors according to: https://arxiv.org/abs/1402.2859
    '''

    rho = ctms[0].shape[0]

    mps = [ctms[2], ctms[5], ctms[3]]
    mpo = [ctms[6], site_tensor, ctms[7]]

    mpo_mps = [None]*3
    mpo_mps[0] = torch.einsum('ABC,aB->aCA', mpo[0], mps[0])
    mpo_mps[1] = torch.einsum('ABCD,efB->eAfCD', mpo[1], mps[1])
    mpo_mps[2] = torch.einsum('ABC,aB->aCA', mpo[2], mps[2])

    # find symmetric projectors
    sym_mpo_mps = deepcopy(mpo_mps)
    sym_mpo_mps.insert(1, mpo_mps[1])

    rs, ls = [], []
    temp = sym_mpo_mps[0]
    q, r = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 0))
    rs.append(r)
    temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[1])
    q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
    rs.append(r)
    temp = torch.einsum('abc,bcdef->adef', r, sym_mpo_mps[2])
    q, r = tp.linalg.tqr(temp, group_dims=((0, 3), (1, 2)), qr_dims=(1, 0))
    rs.append(r)

    temp = sym_mpo_mps[-1]
    q, l = tp.linalg.tqr(temp, group_dims=((2,), (0, 1)), qr_dims=(1, 2))
    ls.append(l)
    temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-2], l)
    q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
    ls.append(l)
    temp = torch.einsum('abcde,cdf->abfe', sym_mpo_mps[-3], l)
    q, l = tp.linalg.tqr(temp, group_dims=((2, 3), (0, 1)), qr_dims=(0, 2))
    ls.append(l)

    ls.reverse()

    u, s, v = tp.linalg.svd(torch.einsum('abc,bcd->ad', rs[1], ls[1]))
    ut, st, vt = u[:, :rho], s[:rho], v[:rho, :]
    ut_dagger, vt_dagger = ut.t().conj(), vt.t().conj()
    # inverse of square root
    sst_inv = (1.0 / torch.sqrt(st)).diag()
    # projectors
    pr = torch.einsum('abc,cd->abd', ls[1], vt_dagger @ sst_inv)
    pl = torch.einsum('ab,bcd->acd', sst_inv @ ut_dagger, rs[1])

    mps = [None]*3
    mps[0] = torch.einsum('abc,abd->dc', mpo_mps[0], pr)
    mps[1] = torch.einsum('abc,bcdef,deg->agf', pl, mpo_mps[1], pr)
    mps[2] = torch.einsum('abc,bcd->ad', pl, mpo_mps[2])

    # update CTM tensors
    ctms[2] = mps[0] / torch.linalg.norm(mps[0])
    ctms[5] = mps[1] / torch.linalg.norm(mps[1])
    ctms[3] = mps[2] / torch.linalg.norm(mps[2])

    return ctms

def ising_free_energy(beta, ctms):

    # bond weight matrix
    w = torch.tensor([[torch.exp(-beta*J), torch.exp(beta*J)], [torch.exp(beta*J), torch.exp(-beta*J)]])
    u, s, v = tp.linalg.svd(w)
    m, mp = u @ torch.sqrt(s).diag(), torch.sqrt(s).diag() @ v

    # build the site tensor
    site_tensor = torch.einsum('as,sb,sc,ds->abcd', mp, m, m, mp)

    ising = SquareClassicalIsing(beta=beta, J=1.0)
    ising.update_ctms(ctms)

    num = 8

    for l in range(num):

        ising.ctmrg_mu_sym()
        ising.ctmrg_md_sym()
        ising.ctmrg_ml_sym()
        ising.ctmrg_mr_sym()

        # bond_ene = ising.ctm_bond_energy()
        # print(l, bond_ene.item()*2)

    # after num times of RG
    f = -1.0*torch.log(ising.parf())

    return f, ising._ctms

def free_energy(beta, ctms):

    ising = SquareClassicalIsing(beta=beta, J=1.0)
    ising.update_ctms(ctms)

    num = 8

    for l in range(num):

        ising.ctmrg_mu_sym()
        ising.ctmrg_md_sym()
        ising.ctmrg_ml_sym()
        ising.ctmrg_mr_sym()

        # bond_ene = ising.ctm_bond_energy()
        # print(l, bond_ene.item()*2)

    # after num times of RG
    f = -1.0*torch.log(ising.parf())

    return f, ising._ctms

f, ctms = free_energy(beta, ctms)
print(f, beta)
# f.backward(inputs=beta)

RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment.  If you were attempting to deepcopy a module, this may be because of a torch.nn.utils.weight_norm usage, see https://github.com/pytorch/pytorch/pull/103001